<a href="https://colab.research.google.com/github/gonzalobarrazueta/TF-20181e098-20191a972-201920156/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imports**

In [ ]:
# Perlin Noise
!pip install perlin_noise
import matplotlib.pyplot as plt
import numpy as np
from perlin_noise import PerlinNoise
# ------------
import csv
import json
# Sección Aristas
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import random
# Dijkstra/Priority Queue
import heapq as hq
import math
# ------------
import numpy as np
import graphviz as gv #sirve para dibujar grafos

##**Diccionario de calles**
Crea un diccionario donde cada nombre de calle ***(key)*** tiene asignado un `id` ***(value)***

In [ ]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [ ]:
def printDictionary(dictionary):
    for k, v in dictionary.items():
        print(f"{k}: {v}")

In [ ]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        for id, row in enumerate(reader):
            addKey_Values(dictionary, row[0].lower(), id)

In [ ]:
def createStreetsDictionary():   
    streets = {}
    cargarDatos(streets)
    return streets

In [ ]:
streetsDictionary = createStreetsDictionary()

##**Lista de Adyacencia de calles y sus intersecciones**




In [ ]:
def printAdList(graph): 
    for i, intersection in enumerate(graph):
        print(i, intersection)
    
    print(f"Cantidad de vértices = {len(graph)}")

In [ ]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        id = streetsDictionary[streetName]
    else:
        id = -1 # la calle no está mapeada
    return id

In [ ]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))
            adjacencyList.append(intersections)

    return adjacencyList

In [ ]:
# creal la lista de adyacencia
intersectionsAdList = createAdjacencyList([])

##**Lista de Adyacencia de intersecciones (Grafo Principal)**

In [ ]:
def createKeys(graph):
    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            # si las keys aún no han sido creadas, las agrega al diccionario
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [ ]:
def addIntersections(graph, i, j):  
    street = graph[i][j]

    if street != -1:
        # agrego la intersección már próxima hacia la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego la intersección már próxima hacia la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [ ]:
def addIntersectionsReversed(graph, i, j):  
    street = graph[i][j]

    # ahora voy al reverso de la lista
    if i != -1 and street != -1:
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [ ]:
def joinIntersections(graph):
    n = len(graph)
    for i in range(n):        
        for j in range(len(graph[i])):
            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [ ]:
# calcula el total de nodos solo par el grafo "map"
def totalNodes(graph):
    total = 0 
    for key, value in graph.items():
        total += len(value)
    return total

In [ ]:
# grafo principal
map = createKeys(intersectionsAdList)
joinIntersections(intersectionsAdList)

##**Diccionario de Latitudes, Longitudes y `ids` para cada intersección**

In [ ]:
def formatearLatitudeorLongitude(input):
    separar = input
    separar = separar.split(".");

    union= separar[0]+"."+separar[1]+separar[2];
    variable_float = float(union)

    return variable_float

In [ ]:
# devuelve solo la latitud y longitud, no el id
def getOnlyLatLong(key):
    return (latlong[key][0], latlong[key][1])

In [ ]:
def getInterId(key):
    return latlong[key][2]

In [ ]:
def createLatLongDictionary(latlong):
    with open("Latitud y Longitud de calles.csv") as file: 
        reader = csv.reader(file)      
        # ignora el header (la 1era fila del csv)
        next(reader)
        # se le asignará un id a cada vértice
        id = 0
        for row in reader:
            # busco el id de la calle en mi diccionario de calles 
            intersection = streetExists(row[0]), streetExists(row[1])
            if intersection in map:
                latlong[intersection] = formatearLatitudeorLongitude(row[2]), formatearLatitudeorLongitude(row[3]), id
                id += 1
    return latlong

In [ ]:
latlong = createLatLongDictionary({})

##**Agregar valores aleatorios**
Agregar valores aleatorios al grafo principal (`map`). Los valores random fueron generados con Perlin Noise

In [ ]:
def getGraphSize(graph):
    return len(graph)

In [ ]:
def perlinNoise():
    # tamaño de cada de arreglo
    xpix = 10
    # cantidad de arreglos
    ypix = getGraphSize(map)
    noise = PerlinNoise(octaves=5, seed=1)
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    return np.array(np.array(pic)*100, dtype=int)

randValues = perlinNoise()

In [ ]:
def sameNode(n1, n2):
    x1, y1 = n1
    x2, y2 = n2

    if (x1 == y2) and (y1 == x2):
        return True
    return False

In [ ]:
def getListRandValues(n):
    r = random.randrange(0, n)
    
    while sum(randValues[r]) == 0:
        r = random.randrange(0, n)

    return list(randValues[r])

In [ ]:
def getPositive(num):
    if num < 0:
        return num * -1
    else:
        return num

In [ ]:
# agrega un valor random (generado con perlin noise) al diccionario de lat y longs
def addRandValue(dictionary):

    randv = getListRandValues(len(latlong))

    for key, value in map.items():
        l = []
        for u in value:
            if sameNode(key, u):
                u += (0,)      
                l.append(u)
                continue

            if len(randv) > 0:
                u += (getPositive(randv.pop()),)
            else:
                randv = getListRandValues(len(latlong))
                u += (getPositive(randv.pop()),)
            l.append(u)
        dictionary[key] = l
    return dictionary
    
map2 = addRandValue({})

##**Escribir archivos**

In [ ]:
def writeNodes():
    with open("nodes.txt", "w") as f:
        with open('Calles de San Martín de Porres.csv') as file: 
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):                    
                    f.write(f"{row[0]}, {row[i]}\n")
writeNodes()

In [ ]:
# Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")
writeStreetsDictionary()

In [ ]:
# Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])    
            f.write(', '.join(streets) + "\n")        
writeStreetsAdList()

In [ ]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:  
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
writeStreetMap()

##**Aristas**

In [ ]:
# devuelve el tiempo actual automáticamente
def getCurrentTime():

    now = datetime.now()
    time = now.strftime("%H %p")
    time = time.split(" ")

    # convierto la hora a la timezone de Perú
    if time[0] <= "5":
        time[0] = 24 - (5 - int(time[0]))
    else:
        time[0] = int(time[0]) - 5
    
    return str(time[0])

def askTime():
    time = int(input("Digite la hora en formato de 24hrs: "));
    if time not in range(0, 24):
        time = askTime() 
    return time

In [ ]:
def getlongitude(n1, n2):
    lon1,lat1=n1
    lon2,lat2=n2
    if lon1 == lat2 and lon2 == lat1:
      longitude = 0
    else:
      lon1 = radians(lon1)
      lon2 = radians(lon2)
      lat1 = radians(lat1)
      lat2 = radians(lat2)
      dlon = lon2 - lon1
      dlat = lat2 - lat1
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
      c = 2 * asin(sqrt(a)) 
      r = 6371
      longitude = c * r
    
    return longitude

In [ ]:
def point_percent(time):
    percent = 0
    # de 0am - 4am
    if time in range(0, 5):
        percent = 0.1
    
    # de 5am - 8am
    if time in range(5, 9):
        percent = 0.9

    # de 9am - 16pm
    if time in range(9, 18):
        percent = 0.5

    # de 18pm - 20pm
    if time in range(18, 21):
        percent = 1.3

    # de 21pm - 23pm
    if time in range(21, 23):
        percent = 0.3

    return percent

In [ ]:
def getFactor(time, val):
    # el factor está en el rango de 0 - 50
    # dependiendo de la hora, mi valor va estar en un determinado rango
    # puntos por hora
    # índice: hora, elemento: puntaje
                 # 0  1  2  3  4  5  6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  
    time_points = [0, 1, 1, 1, 2, 3, 15, 19, 17, 14, 13, 10, 10, 11, 12, 13, 14, 16, 17, 20, 19, 15, 4, 2]

    percent = point_percent(time)
    factor = int((val * percent) + time_points[time])

    if factor > 50:
        factor = 50

    return factor

In [ ]:
def calculateWeight(n1, n2, time, val):
    
    longitude = getlongitude(n1, n2) * 100
    factor = getFactor(time, val)
    
    weight = longitude * factor

    if longitude > 0:
        weight = round(factor + longitude, 2)
    
    if (weight > 100):
        return 99
    else:
        return int(weight)

In [ ]:
# el nuevo grafo será un diccionario, donde cada "key" contiene 1 arreglo de arreglos
# cada arreglo dentro del principal, tiene solo 2 posiciones
# 1era posición del arreglo: 1 tuple de intersecciones
# 2da posición del arreglo: peso de la arista con respecto al valor de la "key"

def updateGraph():
    time = askTime()

    # map con pesos
    mapW = {}
    for key in map2:
        if key in latlong:
            k = getInterId(key)
            mapW[k] = []
        else:
            continue
        for arista in map2[key]:
            edge = (arista[0], arista[1])
            if edge in latlong:
                weight = calculateWeight(getOnlyLatLong(key), getOnlyLatLong(edge), time, arista[2])
                # [intersección, peso]
                if weight == 0: continue
                mapW[k].append((getInterId(edge), weight))
        mapW[k]
    return mapW

In [ ]:
def getMaxKey(G):
    return max(G, key = G.get)

In [ ]:
mapW = updateGraph()

In [178]:
printDictionary(mapW)

0: [(1, 55)]
1: [(0, 55), (2, 27)]
2: [(1, 27), (3, 71), (7, 23)]
3: [(2, 71), (28, 3), (30, 2)]
4: [(1, 55)]
5: [(0, 55), (2, 27)]
6: [(7, 23), (1, 27), (3, 71)]
7: [(6, 23), (8, 39), (26, 37), (28, 61)]
8: [(7, 39), (9, 19), (65, 39)]
9: [(8, 19), (10, 24), (73, 27)]
10: [(9, 24), (11, 5), (55, 79), (57, 66)]
11: [(10, 5), (12, 12)]
12: [(11, 13), (13, 6)]
13: [(12, 6), (14, 13)]
14: [(13, 13), (15, 17)]
15: [(14, 17), (16, 34), (82, 40)]
16: [(15, 35), (17, 14), (84, 62)]
17: [(16, 14), (18, 10), (93, 66)]
18: [(17, 10), (19, 14)]
19: [(18, 14), (20, 11), (98, 5)]
20: [(19, 10), (21, 25)]
21: [(20, 25), (22, 14), (108, 12)]
22: [(21, 14), (23, 43)]
23: [(22, 43), (24, 2)]
24: [(23, 2), (25, 67), (111, 14)]
25: [(24, 67)]
26: [(27, 37), (55, 36)]
27: [(26, 37), (28, 61), (6, 23), (8, 39)]
28: [(27, 63), (29, 5), (32, 42)]
29: [(28, 5), (30, 4), (2, 73)]
30: [(29, 4)]
31: [(32, 42), (27, 63), (29, 5)]
32: [(31, 44), (33, 15)]
33: [(32, 15), (34, 9)]
34: [(33, 9), (35, 24)]
35: [(34, 2

##**Shortest Paths**
Encuentra el camino más corto entre 2 puntos

In [ ]:
def getMin(a):
    min = a[0][1]
    index = 0
    for i, c in enumerate(a):
        if c[1] < min:
            min = c[1]
            index = i
    return a.pop(index)

In [ ]:
# d: destination
# p: path
def definePath(d, path):
    g = []
    g.append(d)

    while path[d] != -1:
        destination = path[d]
        g.insert(0, destination)
        d = destination
    return g

In [ ]:
# uses dijkstra
def findShortestPath(graph, s, ignore):
    n = len(graph)
    path = [-1] * n
    visited = [False] * n
    visited[ignore] = True
    
    cost = [math.inf] * n
    cost[s] = 0

    # priority queue
    # vértice, peso
    priorityq = [(s, 0)]

    while priorityq:
        # vértice, peso
        u, g = getMin(priorityq)
        if not visited[u]:
            visited[u] = True
        for v, w in graph[u]:
            if not visited[v]:
                weight = g + w
                if weight < cost[v]:
                    cost[v] = weight
                    path[v] = u
                    priorityq.append((v, weight))
    return path, cost

In [170]:
# s: source
# d: destiny
def getDirections(s, d, graph):
    neighbours = graph[s]
    paths = []
    costs = []

    for n in neighbours:
        path, cost = findShortestPath(mapW, n[0], s)
        path = definePath(d, path)
        path.insert(0, s)
        paths.append(path)
        costs.append(cost[d])
    return paths, costs

In [221]:
def getBestPaths(paths, costs):
    bestCosts = [(i, x) for i, x in enumerate(costs)]
    bestCosts.sort(key=lambda x:x[1])
    
    n = len(bestCosts)
    if n > 3:
        bestCosts = [bestCosts[i] for i in range(3)]
    else:
        bestCosts = [bestCosts[i] for i in range(n)]

    bestPaths = [paths[c[0]] for c in bestCosts]
    
    return bestPaths

In [222]:
# si el costo es inf, significa que no existe un camino
def aPathExists(paths, costs):
    for i, cost in enumerate(costs):
        if cost == float('Inf'):
            paths[i] = []

In [223]:
def deleteInvalidPaths(paths, costs):
    for i, p in enumerate(paths):
        if p == []:
            paths.pop(i)
            costs.pop(i)

In [228]:
# desde donde quiero empezar a buscar
source = 10
# a donde quiero llegar
destination = 27
paths, costs = getDirections(source, destination, mapW)

# elimina caminos inválidos (no existentes)
aPathExists(paths, costs)
deleteInvalidPaths(paths, costs)

# retorna los mejores caminos
paths = getBestPaths(paths, costs)
costs.sort()

print(paths, costs)

[[10, 55, 54, 27], [10, 57, 75, 121, 120, 65, 54, 27], [10, 9, 8, 7, 26, 27]] [73, 108, 132]


##**Visualizaciones**
Aquí se encuentra todo el código para escribir los datos en el formato necesario para que puedan ser graficados.

In [ ]:
def latlongGraph(dictionary):
    n = len(dictionary) + 10
    latlongs = [()] * n
    
    for key, value in dictionary.items():
        latlongs[dictionary[key][2]] = dictionary[key][0] * -1, dictionary[key][1] * -1

    return latlongs

In [ ]:
def getGraph(graph):
    n = len(graph) + 10
    G = [-1] * n

    for key, value in graph.items():
        G[key] = value
    return G

In [ ]:
# loc, G
latlongs = latlongGraph(latlong); graph = getGraph(mapW)